In [ ]:
import data_cleaning as dc
import review_score_analysis as rs
import text_analysis as ml
import pickle

In [2]:
# Load dataframes
chunk_size = 100_000
restaurants_df = dc.load("data/filtered_restaurants.json", chunk_size)
reviews_df = dc.load("data/filtered_reviews.json", chunk_size)
avg_scores_df = rs.calculate_average_review_score(reviews_df)

In [ ]:
avg_scores_df.info()


In [3]:
# Test functions
ml.train_evaluate_model(avg_scores_df, 50000)

Benchmark accuracy for our model to beat: 0.94802
Accuracy of our model: 0.9989


In [3]:
X, y, classifier = ml.train_model(avg_scores_df, 500_000)

/home/chasty2/Documents/group-project-better-late-than-never/text_analysis.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(process)


In [6]:
with open("data/2_label_svc.pkl", "wb") as file:
    pickle.dump(classifier, file)